In [1]:
# 📦 Gerekli kütüphaneleri yükleyelim
import os
import json
import pickle
import joblib
import requests
import time
from datetime import datetime
from pathlib import Path

# Data processing
import pandas as pd
import numpy as np

# ML libraries
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# MLflow
import mlflow
import mlflow.sklearn

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Web framework
from flask import Flask, request, jsonify
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import threading

# Testing ve monitoring
from concurrent.futures import ThreadPoolExecutor
import psutil

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Plotting ayarları
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("📦 Tüm kütüphaneler başarıyla yüklendi!")
print("🚀 Model Deployment eğitimine hazırız!")
print(f"📅 Başlangıç zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📦 Tüm kütüphaneler başarıyla yüklendi!
🚀 Model Deployment eğitimine hazırız!
📅 Başlangıç zamanı: 2025-07-02 17:21:39


In [ ]:
# 🔧 Düzeltilmiş Model Packaging Kodu
class ModelPackager:
    """Model packaging ve versioning için sınıf"""
    
    def __init__(self, model_dir="models"):
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        self.metadata = {}
    
    def prepare_model(self):
        """Demo için basit bir model hazırla"""
        print("🏠 California Housing veri seti ile model eğitiliyor...")
        
        # Veri yükleme
        california_housing = fetch_california_housing()
        X = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
        y = california_housing.target
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Preprocessing
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Model eğitimi
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X_train_scaled, y_train)
        
        # Test performansı
        y_pred = self.model.predict(X_test_scaled)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = r2_score(y_test, y_pred)
        
        # Metadata (düzeltilmiş - .tolist() hatası giderildi)
        self.metadata = {
            'model_type': 'RandomForestRegressor',
            'version': '1.0.0',
            'training_date': datetime.now().isoformat(),
            'test_rmse': test_rmse,
            'test_r2': test_r2,
            'feature_names': list(california_housing.feature_names),  # Düzeltme burada
            'input_shape': X.shape[1],
            'target_name': 'house_value'
        }
        
        print(f"✅ Model eğitildi!")
        print(f"📊 Test RMSE: {test_rmse:.4f}")
        print(f"📊 Test R²: {test_r2:.4f}")
        
        return X_test, y_test
    
    def save_model(self, version="v1"):
        """Modeli ve preprocessing pipeline'ını kaydet"""
        version_dir = self.model_dir / version
        version_dir.mkdir(exist_ok=True)
        
        # Model kaydet
        model_path = version_dir / "model.pkl"
        joblib.dump(self.model, model_path)
        
        # Scaler kaydet
        scaler_path = version_dir / "scaler.pkl"
        joblib.dump(self.scaler, scaler_path)
        
        # Metadata kaydet
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(self.metadata, f, indent=2)
        
        print(f"💾 Model kaydedildi: {version_dir}")
        print(f"   📁 Model: {model_path}")
        print(f"   📁 Scaler: {scaler_path}")
        print(f"   📁 Metadata: {metadata_path}")
        
        return version_dir
    
    def load_model(self, version="v1"):
        """Kaydedilmiş modeli yükle"""
        version_dir = self.model_dir / version
        
        if not version_dir.exists():
            raise FileNotFoundError(f"Model version {version} bulunamadı!")
        
        # Model yükle
        model_path = version_dir / "model.pkl"
        self.model = joblib.load(model_path)
        
        # Scaler yükle
        scaler_path = version_dir / "scaler.pkl"
        self.scaler = joblib.load(scaler_path)
        
        # Metadata yükle
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        
        print(f"📥 Model yüklendi: {version}")
        print(f"📊 Model Type: {self.metadata['model_type']}")
        print(f"📊 Version: {self.metadata['version']}")
        print(f"📊 Test R²: {self.metadata['test_r2']:.4f}")
        
        return self.model, self.scaler, self.metadata
    
    def predict(self, X):
        """Tahmin yap"""
        if hasattr(X, 'values'):
            X = X.values
        
        X_scaled = self.scaler.transform(X)
        predictions = self.model.predict(X_scaled)
        
        return predictions

# Model paketi oluştur (düzeltilmiş sürüm)
packager = ModelPackager()
X_test, y_test = packager.prepare_model()
model_version_dir = packager.save_model("v1")

print("\n✅ Model packaging tamamlandı!")
print(f"📁 Model dizini: {model_version_dir}")
print(f"📊 Feature adları: {packager.metadata['feature_names']}")
print(f"📊 Input shape: {packager.metadata['input_shape']}")


In [3]:
class ModelPackager:
    """Model packaging ve versioning için sınıf"""
    
    def __init__(self, model_dir="models"):
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        self.metadata = {}
    
    def prepare_model(self):
        """Demo için basit bir model hazırla"""
        print("🏠 California Housing veri seti ile model eğitiliyor...")
        
        # Veri yükleme
        california_housing = fetch_california_housing()
        X = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
        y = california_housing.target
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Preprocessing
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Model eğitimi
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X_train_scaled, y_train)
        
        # Test performansı
        y_pred = self.model.predict(X_test_scaled)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = r2_score(y_test, y_pred)
        
        # Metadata
        self.metadata = {
            'model_type': 'RandomForestRegressor',
            'version': '1.0.0',
            'training_date': datetime.now().isoformat(),
            'test_rmse': test_rmse,
            'test_r2': test_r2,
            'feature_names': california_housing.feature_names.tolist(),
            'input_shape': X.shape[1],
            'target_name': 'house_value'
        }
        
        print(f"✅ Model eğitildi!")
        print(f"📊 Test RMSE: {test_rmse:.4f}")
        print(f"📊 Test R²: {test_r2:.4f}")
        
        return X_test, y_test
    
    def save_model(self, version="v1"):
        """Modeli ve preprocessing pipeline'ını kaydet"""
        version_dir = self.model_dir / version
        version_dir.mkdir(exist_ok=True)
        
        # Model kaydet
        model_path = version_dir / "model.pkl"
        joblib.dump(self.model, model_path)
        
        # Scaler kaydet
        scaler_path = version_dir / "scaler.pkl"
        joblib.dump(self.scaler, scaler_path)
        
        # Metadata kaydet
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(self.metadata, f, indent=2)
        
        print(f"💾 Model kaydedildi: {version_dir}")
        print(f"   📁 Model: {model_path}")
        print(f"   📁 Scaler: {scaler_path}")
        print(f"   📁 Metadata: {metadata_path}")
        
        return version_dir
    
    def load_model(self, version="v1"):
        """Kaydedilmiş modeli yükle"""
        version_dir = self.model_dir / version
        
        if not version_dir.exists():
            raise FileNotFoundError(f"Model version {version} bulunamadı!")
        
        # Model yükle
        model_path = version_dir / "model.pkl"
        self.model = joblib.load(model_path)
        
        # Scaler yükle
        scaler_path = version_dir / "scaler.pkl"
        self.scaler = joblib.load(scaler_path)
        
        # Metadata yükle
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        
        print(f"📥 Model yüklendi: {version}")
        print(f"📊 Model Type: {self.metadata['model_type']}")
        print(f"📊 Version: {self.metadata['version']}")
        print(f"📊 Test R²: {self.metadata['test_r2']:.4f}")
        
        return self.model, self.scaler, self.metadata
    
    def predict(self, X):
        """Tahmin yap"""
        if hasattr(X, 'values'):
            X = X.values
        
        X_scaled = self.scaler.transform(X)
        predictions = self.model.predict(X_scaled)
        
        return predictions

# Model paketi oluştur
packager = ModelPackager()
X_test, y_test = packager.prepare_model()
model_version_dir = packager.save_model("v1")

print("\n✅ Model packaging tamamlandı!")


🏠 California Housing veri seti ile model eğitiliyor...


AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
class ModelService:
    """Model servisi için sınıf"""
    
    def __init__(self, model_version="v1"):
        self.packager = ModelPackager()
        self.model, self.scaler, self.metadata = self.packager.load_model(model_version)
        self.prediction_count = 0
        self.start_time = time.time()
    
    def predict_single(self, features):
        """Tek tahmin yap"""
        try:
            # Feature validation
            if len(features) != self.metadata['input_shape']:
                raise ValueError(f"Expected {self.metadata['input_shape']} features, got {len(features)}")
            
            # Prediction
            X = np.array(features).reshape(1, -1)
            prediction = self.packager.predict(X)[0]
            
            self.prediction_count += 1
            
            return {
                'prediction': float(prediction),
                'model_version': self.metadata['version'],
                'timestamp': datetime.now().isoformat(),
                'prediction_id': self.prediction_count
            }
        
        except Exception as e:
            return {
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }
    
    def predict_batch(self, features_list):
        """Batch tahmin yap"""
        try:
            predictions = []
            for features in features_list:
                result = self.predict_single(features)
                predictions.append(result)
            
            return {
                'predictions': predictions,
                'batch_size': len(features_list),
                'model_version': self.metadata['version'],
                'timestamp': datetime.now().isoformat()
            }
        
        except Exception as e:
            return {
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }
    
    def get_health(self):
        """Health check"""
        uptime = time.time() - self.start_time
        
        return {
            'status': 'healthy',
            'uptime_seconds': uptime,
            'total_predictions': self.prediction_count,
            'model_version': self.metadata['version'],
            'model_type': self.metadata['model_type'],
            'timestamp': datetime.now().isoformat()
        }
    
    def get_model_info(self):
        """Model bilgilerini döndür"""
        return {
            'metadata': self.metadata,
            'service_stats': {
                'uptime_seconds': time.time() - self.start_time,
                'total_predictions': self.prediction_count
            }
        }

# Model servisini başlat
model_service = ModelService("v1")

print("🚀 Model servisi hazır!")
print(f"📊 Model: {model_service.metadata['model_type']}")
print(f"📊 Version: {model_service.metadata['version']}")


In [ ]:
# Pydantic modelleri
class PredictionRequest(BaseModel):
    features: list
    
class BatchPredictionRequest(BaseModel):
    features_list: list

# FastAPI app oluştur
def create_fastapi_app():
    app = FastAPI(
        title="🏠 California Housing Price Predictor",
        description="Machine Learning model service for house price prediction",
        version="1.0.0"
    )
    
    # Global model service
    service = ModelService("v1")
    
    @app.get("/")
    async def root():
        return {
            "message": "🏠 California Housing Price Predictor API",
            "version": "1.0.0",
            "model_version": service.metadata['version'],
            "docs": "/docs"
        }
    
    @app.get("/health")
    async def health_check():
        return service.get_health()
    
    @app.get("/model/info")
    async def model_info():
        return service.get_model_info()
    
    @app.post("/predict")
    async def predict(request: PredictionRequest):
        result = service.predict_single(request.features)
        if 'error' in result:
            raise HTTPException(status_code=400, detail=result['error'])
        return result
    
    @app.post("/predict/batch")
    async def predict_batch(request: BatchPredictionRequest):
        result = service.predict_batch(request.features_list)
        if 'error' in result:
            raise HTTPException(status_code=400, detail=result['error'])
        return result
    
    return app

# API app'i oluştur
api_app = create_fastapi_app()

print("🌐 FastAPI application oluşturuldu!")
print("📖 API Documentation: http://localhost:8000/docs")
print("🏥 Health Check: http://localhost:8000/health")


In [ ]:
# API testi için örnek veriler hazırlayalım
class APITester:
    """API test sınıfı"""
    
    def __init__(self, service):
        self.service = service
        self.test_results = {}
    
    def test_single_prediction(self):
        """Tek tahmin testi"""
        print("🔍 Tek tahmin testi...")
        
        # Test verisinden örnek al
        sample_features = X_test.iloc[0].tolist()
        actual_value = y_test.iloc[0]
        
        # Tahmin yap
        result = self.service.predict_single(sample_features)
        
        if 'error' not in result:
            predicted_value = result['prediction']
            error = abs(predicted_value - actual_value)
            
            print(f"✅ Tek tahmin başarılı!")
            print(f"   📊 Gerçek değer: ${actual_value:.2f}00k")
            print(f"   📊 Tahmin değer: ${predicted_value:.2f}00k")
            print(f"   📊 Hata: ${error:.2f}00k")
            print(f"   🆔 Prediction ID: {result['prediction_id']}")
            
            self.test_results['single_prediction'] = {
                'success': True,
                'actual': actual_value,
                'predicted': predicted_value,
                'error': error
            }
        else:
            print(f"❌ Tek tahmin hatası: {result['error']}")
            self.test_results['single_prediction'] = {'success': False, 'error': result['error']}
    
    def test_batch_prediction(self, batch_size=5):
        """Batch tahmin testi"""
        print(f"\n🔍 Batch tahmin testi ({batch_size} örnek)...")
        
        # Test verisinden batch al
        batch_features = X_test.iloc[:batch_size].values.tolist()
        actual_values = y_test.iloc[:batch_size].tolist()
        
        # Batch tahmin yap
        result = self.service.predict_batch(batch_features)
        
        if 'error' not in result:
            predictions = [pred['prediction'] for pred in result['predictions']]
            errors = [abs(pred - actual) for pred, actual in zip(predictions, actual_values)]
            mean_error = np.mean(errors)
            
            print(f"✅ Batch tahmin başarılı!")
            print(f"   📊 Batch size: {result['batch_size']}")
            print(f"   📊 Ortalama hata: ${mean_error:.2f}00k")
            print(f"   📊 Min hata: ${min(errors):.2f}00k")
            print(f"   📊 Max hata: ${max(errors):.2f}00k")
            
            self.test_results['batch_prediction'] = {
                'success': True,
                'batch_size': batch_size,
                'mean_error': mean_error,
                'predictions': predictions,
                'actual_values': actual_values
            }
        else:
            print(f"❌ Batch tahmin hatası: {result['error']}")
            self.test_results['batch_prediction'] = {'success': False, 'error': result['error']}
    
    def test_health_check(self):
        """Health check testi"""
        print(f"\n🏥 Health check testi...")
        
        health = self.service.get_health()
        
        print(f"✅ Health check sonuçları:")
        print(f"   📊 Status: {health['status']}")
        print(f"   📊 Uptime: {health['uptime_seconds']:.2f} saniye")
        print(f"   📊 Total predictions: {health['total_predictions']}")
        print(f"   📊 Model version: {health['model_version']}")
        
        self.test_results['health_check'] = health
    
    def test_model_info(self):
        \"\"\"Model info testi\"\"\"
        print(f\"\\n📋 Model info testi...\")
        
        info = self.service.get_model_info()
        
        print(f\"✅ Model bilgileri:\")
        print(f\"   📊 Model type: {info['metadata']['model_type']}\")
        print(f\"   📊 Version: {info['metadata']['version']}\")
        print(f\"   📊 Training date: {info['metadata']['training_date'][:10]}\")
        print(f\"   📊 Test R²: {info['metadata']['test_r2']:.4f}\")
        print(f\"   📊 Input features: {info['metadata']['input_shape']}\")
        
        self.test_results['model_info'] = info
    
    def test_error_handling(self):
        \"\"\"Error handling testi\"\"\"
        print(f\"\\n⚠️ Error handling testi...\")
        
        # Yanlış feature sayısı ile test
        wrong_features = [1, 2, 3]  # 8 olması gerekiyor
        result = self.service.predict_single(wrong_features)
        
        if 'error' in result:
            print(f\"✅ Error handling çalışıyor: {result['error']}\")
            self.test_results['error_handling'] = {'success': True, 'error_message': result['error']}
        else:
            print(f\"❌ Error handling çalışmıyor!\")
            self.test_results['error_handling'] = {'success': False}
    
    def run_all_tests(self):
        \"\"\"Tüm testleri çalıştır\"\"\"
        print(\"🧪 API TESTLER BAŞLATIYOR...\")
        print(\"=\" * 50)
        
        self.test_single_prediction()
        self.test_batch_prediction()
        self.test_health_check()
        self.test_model_info()
        self.test_error_handling()
        
        print(\"\\n\" + \"=\" * 50)
        print(\"✅ Tüm testler tamamlandı!\")
        
        return self.test_results

# API testlerini çalıştır
tester = APITester(model_service)
test_results = tester.run_all_tests()


In [ ]:
class DockerDeployment:
    """Docker deployment sınıfı"""
    
    def __init__(self, app_name="housing-predictor"):
        self.app_name = app_name
        self.dockerfile_content = None
        self.requirements_content = None
        self.app_content = None
    
    def generate_requirements(self):
        """Requirements.txt dosyası oluştur"""
        self.requirements_content = """fastapi==0.104.1
uvicorn==0.24.0
pandas==2.1.4
numpy==1.24.3
scikit-learn==1.3.2
joblib==1.3.2
pydantic==2.5.0
"""
        
        # Requirements dosyasını yaz
        with open("requirements.txt", "w") as f:
            f.write(self.requirements_content)
        
        print("📦 requirements.txt oluşturuldu!")
        return self.requirements_content
    
    def generate_app_file(self):
        """Standalone app.py dosyası oluştur"""
        self.app_content = '''import os
import json
import joblib
import numpy as np
from pathlib import Path
from datetime import datetime
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

# Pydantic modelleri
class PredictionRequest(BaseModel):
    features: list

class BatchPredictionRequest(BaseModel):
    features_list: list

class ModelService:
    """Model servisi"""
    
    def __init__(self, model_dir="models", version="v1"):
        self.model_dir = Path(model_dir)
        self.version = version
        self.prediction_count = 0
        self.start_time = datetime.now().timestamp()
        self.load_model()
    
    def load_model(self):
        """Modeli yükle"""
        version_dir = self.model_dir / self.version
        
        # Model yükle
        model_path = version_dir / "model.pkl"
        self.model = joblib.load(model_path)
        
        # Scaler yükle
        scaler_path = version_dir / "scaler.pkl"
        self.scaler = joblib.load(scaler_path)
        
        # Metadata yükle
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
    
    def predict(self, features):
        """Tahmin yap"""
        X = np.array(features).reshape(1, -1)
        X_scaled = self.scaler.transform(X)
        prediction = self.model.predict(X_scaled)[0]
        
        self.prediction_count += 1
        
        return {
            'prediction': float(prediction),
            'model_version': self.metadata['version'],
            'timestamp': datetime.now().isoformat(),
            'prediction_id': self.prediction_count
        }

# FastAPI app
app = FastAPI(
    title="🏠 Housing Price Predictor",
    description="ML model service",
    version="1.0.0"
)

# Model servisi
service = ModelService()

@app.get("/")
async def root():
    return {
        "message": "🏠 Housing Price Predictor API",
        "version": "1.0.0",
        "status": "healthy"
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "uptime": datetime.now().timestamp() - service.start_time,
        "predictions": service.prediction_count
    }

@app.post("/predict")
async def predict(request: PredictionRequest):
    try:
        if len(request.features) != service.metadata['input_shape']:
            raise HTTPException(400, f"Expected {service.metadata['input_shape']} features")
        
        return service.predict(request.features)
    except Exception as e:
        raise HTTPException(400, str(e))

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''
        
        # App dosyasını yaz
        with open("app.py", "w") as f:
            f.write(self.app_content)
        
        print("🐍 app.py oluşturuldu!")
        return self.app_content
    
    def generate_dockerfile(self):
        """Dockerfile oluştur"""
        self.dockerfile_content = f"""# Python 3.9 slim image kullan
FROM python:3.9-slim

# Working directory ayarla
WORKDIR /app

# System dependencies
RUN apt-get update && apt-get install -y \\
    gcc \\
    && rm -rf /var/lib/apt/lists/*

# Python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Uygulama dosyalarını kopyala
COPY app.py .
COPY models/ ./models/

# Port expose et
EXPOSE 8000

# Health check ekle
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

# Uygulamayı başlat
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
"""
        
        # Dockerfile'ı yaz
        with open("Dockerfile", "w") as f:
            f.write(self.dockerfile_content)
        
        print("🐳 Dockerfile oluşturuldu!")
        return self.dockerfile_content
    
    def generate_docker_compose(self):
        """Docker Compose dosyası oluştur"""
        compose_content = f"""version: '3.8'

services:
  {self.app_name}:
    build: .
    ports:
      - "8000:8000"
    environment:
      - ENV=production
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s
"""
        
        # Docker compose dosyasını yaz
        with open("docker-compose.yml", "w") as f:
            f.write(compose_content)
        
        print("🔧 docker-compose.yml oluşturuldu!")
        return compose_content
    
    def generate_all_files(self):
        """Tüm Docker dosyalarını oluştur"""
        print("🏗️ Docker deployment dosyaları oluşturuluyor...")
        
        self.generate_requirements()
        self.generate_app_file()
        self.generate_dockerfile()
        self.generate_docker_compose()
        
        print("\n✅ Docker deployment hazır!")
        print("🚀 Çalıştırmak için:")
        print("   docker-compose up --build")
        print("📖 API docs: http://localhost:8000/docs")
        
        return {
            'requirements': self.requirements_content,
            'app': self.app_content,
            'dockerfile': self.dockerfile_content
        }

# Docker deployment oluştur
docker_deploy = DockerDeployment("housing-predictor")
docker_files = docker_deploy.generate_all_files()


In [ ]:
class LoadTester:
    """Load testing sınıfı"""
    
    def __init__(self, service):
        self.service = service
        self.test_results = {}
        
    def single_request_benchmark(self, num_requests=100):
        """Tek request benchmark testi"""
        print(f"⚡ Tek request benchmark ({num_requests} istek)...")
        
        # Test verisi hazırla
        sample_features = X_test.iloc[0].tolist()
        
        start_time = time.time()
        successful_requests = 0
        failed_requests = 0
        response_times = []
        
        for i in range(num_requests):
            request_start = time.time()
            
            try:
                result = self.service.predict_single(sample_features)
                request_end = time.time()
                
                if 'error' not in result:
                    successful_requests += 1
                    response_times.append((request_end - request_start) * 1000)  # ms
                else:
                    failed_requests += 1
                    
            except Exception as e:
                failed_requests += 1
        
        total_time = time.time() - start_time
        avg_response_time = np.mean(response_times) if response_times else 0
        requests_per_second = successful_requests / total_time if total_time > 0 else 0
        
        print(f"✅ Tek request benchmark sonuçları:")
        print(f"   📊 Toplam istek: {num_requests}")
        print(f"   📊 Başarılı: {successful_requests}")
        print(f"   📊 Başarısız: {failed_requests}")
        print(f"   📊 Ortalama yanıt süresi: {avg_response_time:.2f} ms")
        print(f"   📊 Min yanıt süresi: {min(response_times):.2f} ms")
        print(f"   📊 Max yanıt süresi: {max(response_times):.2f} ms")
        print(f"   📊 Request/saniye: {requests_per_second:.2f}")
        
        self.test_results['single_request'] = {
            'total_requests': num_requests,
            'successful': successful_requests,
            'failed': failed_requests,
            'avg_response_time_ms': avg_response_time,
            'min_response_time_ms': min(response_times) if response_times else 0,
            'max_response_time_ms': max(response_times) if response_times else 0,
            'requests_per_second': requests_per_second,
            'total_time_seconds': total_time
        }
        
        return self.test_results['single_request']
    
    def concurrent_request_benchmark(self, num_threads=10, requests_per_thread=10):
        """Concurrent request benchmark testi"""
        print(f"🚀 Concurrent benchmark ({num_threads} thread, {requests_per_thread} istek/thread)...")
        
        sample_features = X_test.iloc[0].tolist()
        all_response_times = []
        successful_requests = 0
        failed_requests = 0
        
        def worker():
            nonlocal successful_requests, failed_requests
            thread_response_times = []
            
            for _ in range(requests_per_thread):
                request_start = time.time()
                
                try:
                    result = self.service.predict_single(sample_features)
                    request_end = time.time()
                    
                    if 'error' not in result:
                        successful_requests += 1
                        thread_response_times.append((request_end - request_start) * 1000)
                    else:
                        failed_requests += 1
                        
                except Exception as e:
                    failed_requests += 1
            
            all_response_times.extend(thread_response_times)
        
        start_time = time.time()
        
        # Thread pool ile test çalıştır
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(worker) for _ in range(num_threads)]
            
            # Tüm thread'lerin bitmesini bekle
            for future in futures:
                future.result()
        
        total_time = time.time() - start_time
        total_requests = num_threads * requests_per_thread
        avg_response_time = np.mean(all_response_times) if all_response_times else 0
        requests_per_second = successful_requests / total_time if total_time > 0 else 0
        
        print(f"✅ Concurrent benchmark sonuçları:")
        print(f"   📊 Toplam thread: {num_threads}")
        print(f"   📊 Thread başına istek: {requests_per_thread}")
        print(f"   📊 Toplam istek: {total_requests}")
        print(f"   📊 Başarılı: {successful_requests}")
        print(f"   📊 Başarısız: {failed_requests}")
        print(f"   📊 Ortalama yanıt süresi: {avg_response_time:.2f} ms")
        print(f"   📊 Request/saniye: {requests_per_second:.2f}")
        
        self.test_results['concurrent'] = {
            'num_threads': num_threads,
            'requests_per_thread': requests_per_thread,
            'total_requests': total_requests,
            'successful': successful_requests,
            'failed': failed_requests,
            'avg_response_time_ms': avg_response_time,
            'requests_per_second': requests_per_second,
            'total_time_seconds': total_time
        }
        
        return self.test_results['concurrent']
    
    def memory_usage_test(self, num_predictions=1000):
        """Memory usage testi"""
        print(f"💾 Memory usage testi ({num_predictions} tahmin)...")
        
        # Başlangıç memory kullanımı
        process = psutil.Process()
        initial_memory = process.memory_info().rss / 1024 / 1024  # MB
        
        sample_features = X_test.iloc[0].tolist()
        
        # Tahminler yap
        for i in range(num_predictions):
            result = self.service.predict_single(sample_features)
            
            # Her 100 tahminde bir memory kontrolü
            if i % 100 == 0:
                current_memory = process.memory_info().rss / 1024 / 1024
                print(f"   📊 {i} tahmin - Memory: {current_memory:.2f} MB")
        
        # Final memory kullanımı
        final_memory = process.memory_info().rss / 1024 / 1024
        memory_increase = final_memory - initial_memory
        
        print(f"✅ Memory usage sonuçları:")
        print(f"   📊 Başlangıç memory: {initial_memory:.2f} MB")
        print(f"   📊 Final memory: {final_memory:.2f} MB")
        print(f"   📊 Memory artışı: {memory_increase:.2f} MB")
        print(f"   📊 Tahmin başına memory: {memory_increase/num_predictions*1000:.4f} KB")
        
        self.test_results['memory'] = {
            'initial_memory_mb': initial_memory,
            'final_memory_mb': final_memory,
            'memory_increase_mb': memory_increase,
            'memory_per_prediction_kb': memory_increase/num_predictions*1000,
            'num_predictions': num_predictions
        }
        
        return self.test_results['memory']
    
    def plot_performance_results(self):
        """Performance sonuçlarını görselleştir"""
        if not self.test_results:
            print("❌ Henüz test yapılmamış!")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Response time comparison
        if 'single_request' in self.test_results and 'concurrent' in self.test_results:
            response_times = [
                self.test_results['single_request']['avg_response_time_ms'],
                self.test_results['concurrent']['avg_response_time_ms']
            ]
            test_types = ['Single Thread', 'Concurrent']
            
            axes[0, 0].bar(test_types, response_times, alpha=0.7, color=['blue', 'orange'])
            axes[0, 0].set_title('📊 Average Response Time')
            axes[0, 0].set_ylabel('Response Time (ms)')
            axes[0, 0].grid(True, alpha=0.3)
            
            # Add value labels
            for i, v in enumerate(response_times):
                axes[0, 0].text(i, v + max(response_times)*0.01, f'{v:.2f} ms', 
                               ha='center', va='bottom')
        
        # Requests per second comparison
        if 'single_request' in self.test_results and 'concurrent' in self.test_results:
            rps_values = [
                self.test_results['single_request']['requests_per_second'],
                self.test_results['concurrent']['requests_per_second']
            ]
            
            axes[0, 1].bar(test_types, rps_values, alpha=0.7, color=['green', 'red'])
            axes[0, 1].set_title('🚀 Requests Per Second')
            axes[0, 1].set_ylabel('Requests/Second')
            axes[0, 1].grid(True, alpha=0.3)
            
            # Add value labels
            for i, v in enumerate(rps_values):
                axes[0, 1].text(i, v + max(rps_values)*0.01, f'{v:.2f}', 
                               ha='center', va='bottom')
        
        # Success rate
        if 'single_request' in self.test_results:
            single_success_rate = (self.test_results['single_request']['successful'] / 
                                 self.test_results['single_request']['total_requests'] * 100)
            concurrent_success_rate = (self.test_results['concurrent']['successful'] / 
                                     self.test_results['concurrent']['total_requests'] * 100)
            
            success_rates = [single_success_rate, concurrent_success_rate]
            
            axes[1, 0].bar(test_types, success_rates, alpha=0.7, color=['lightgreen', 'lightcoral'])
            axes[1, 0].set_title('✅ Success Rate')
            axes[1, 0].set_ylabel('Success Rate (%)')
            axes[1, 0].set_ylim(0, 105)
            axes[1, 0].grid(True, alpha=0.3)
            
            # Add value labels
            for i, v in enumerate(success_rates):
                axes[1, 0].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
        
        # Memory usage
        if 'memory' in self.test_results:
            memory_data = self.test_results['memory']
            memory_categories = ['Initial', 'Final', 'Increase']
            memory_values = [
                memory_data['initial_memory_mb'],
                memory_data['final_memory_mb'],
                memory_data['memory_increase_mb']
            ]
            
            bars = axes[1, 1].bar(memory_categories, memory_values, alpha=0.7, 
                                color=['lightblue', 'orange', 'red'])
            axes[1, 1].set_title('💾 Memory Usage')
            axes[1, 1].set_ylabel('Memory (MB)')
            axes[1, 1].grid(True, alpha=0.3)
            
            # Add value labels
            for bar, value in zip(bars, memory_values):
                axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(memory_values)*0.01,
                               f'{value:.2f} MB', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
    
    def run_all_tests(self):
        """Tüm performance testlerini çalıştır"""
        print("⚡ LOAD TESTING BAŞLATIYOR...")
        print("=" * 60)
        
        # Single request benchmark
        self.single_request_benchmark(num_requests=100)
        print()
        
        # Concurrent benchmark
        self.concurrent_request_benchmark(num_threads=5, requests_per_thread=20)
        print()
        
        # Memory usage test
        self.memory_usage_test(num_predictions=500)
        print()
        
        # Results visualization
        self.plot_performance_results()
        
        print("=" * 60)
        print("✅ Tüm performance testleri tamamlandı!")
        
        return self.test_results

# Load testing çalıştır
load_tester = LoadTester(model_service)
performance_results = load_tester.run_all_tests()
